In [1]:
import yaml
from utils.utils import load_config

In [5]:
path = 'models\\configs\\config_ViT.yaml'

In [6]:
list_training = {'original': {'soft_label': 'None', 'label_smoothing': 0}}

In [ ]:
list_training = {'LS_01': {'soft_label': 'None', 'label_smoothing': 0.1}}

In [ ]:
list_training = {'LS_03': {'soft_label': 'None', 'label_smoothing': 0.3}}

In [ ]:
list_training = {'LS_05': {'soft_label': 'None', 'label_smoothing': 0.5}}

In [ ]:
list_training = {'SIGMOID': {'soft_label': 'sigmoid', 'label_smoothing': 0}}

In [3]:
list_training = {'LINEAR': {'soft_label': 'linear', 'label_smoothing': 0}}

In [3]:
list_training = {'STEP': {'soft_label': 'step', 'label_smoothing': 0}}

In [ ]:
list_training = {
                 'LS_01': {'soft_label': 'None', 'label_smoothing': 0.1}, 
                 'LS_03': {'soft_label': 'None', 'label_smoothing': 0.3}, 
                 'LS_05': {'soft_label': 'None', 'label_smoothing': 0.5}, 
                 'SIGMOID': {'soft_label': 'sigmoid', 'label_smoothing': 0}, 
                 'LINEAR': {'soft_label': 'linear', 'label_smoothing': 0}, 
                 'STEP': {'soft_label': 'step', 'label_smoothing': 0}}

# TRAIN CLASSIC

In [ ]:
import time


for trainining_paremeters in list_training.keys():
    
    print(f'Starting training with parameters: {trainining_paremeters}')
    for i in range(0,10):
        print(f'training: {trainining_paremeters} Fold: {i}')

        config = load_config(path)
        config['path_test'] = f'Datasets\Folds\{i}\Test'
        config['path_train'] = f'Datasets\Folds\{i}\Train'
        config['soft_label'] = list_training[trainining_paremeters]['soft_label']
        config['label_smoothing'] = list_training[trainining_paremeters]['label_smoothing']

        with open(path, 'w') as file:
            yaml.dump(config, file)

        !python train.py --config $path


# TRAIN ENSEMBLES

In [ ]:
import time
import yaml

seeds = [1234, 5678, 91011, 121314, 151617, 2023, 31415, 27182, 424242, 8675309]
seeds = [2023]

total_runs = len(list_training.keys()) * len(seeds) * 10
run_counter = 0
start_all = time.time()

for training_parameters in list_training.keys():
    print(f'Starting training with parameters: {training_parameters}')
    for j, seed in enumerate(seeds):
        for i in range(0, 10):  # Folds
            run_counter += 1
            run_start = time.time()

            print(f'[{run_counter}/{total_runs}] Training: {training_parameters} | Fold: {i} | Seed: {seed}')

            config = load_config(path)
            config['path_test'] = f'Datasets\\Folds\\{i}\\Test'
            config['path_train'] = f'Datasets\\Folds\\{i}\\Train'
            config['soft_label'] = list_training[training_parameters]['soft_label']
            config['label_smoothing'] = list_training[training_parameters]['label_smoothing']
            config['seed'] = seed
            config['save_dir'] = f'experiments\\ViT_B_32_ENSEMBLE_{training_parameters}\\ensemble_{j+5}'

            with open(path, 'w') as file:
                yaml.dump(config, file)

            !python train.py --config $path

            run_end = time.time()
            elapsed_run = run_end - run_start
            elapsed_total = run_end - start_all
            avg_per_run = elapsed_total / run_counter
            remaining_time = avg_per_run * (total_runs - run_counter)

            print(f"Run time: {elapsed_run:.2f}s | "
                  f"Total elapsed: {elapsed_total/60:.2f} min | "
                  f"Estimated remaining: {remaining_time/60:.2f} min\n")

# TRAIN FINAL

In [4]:
import time

print(f'Starting training')

config = load_config(path)
config['path_test'] = f'Datasets\Folds\TrainAll\Test'
config['path_train'] = f'Datasets\Folds\TrainAll\Train'
config['soft_label'] = list_training['original']['soft_label']
config['label_smoothing'] = list_training['original']['label_smoothing']

with open(path, 'w') as file:
    yaml.dump(config, file)

!python train.py --config $path


Starting training
Caching images and labels, please wait...
Caching images and labels, please wait...


2025-08-26 13:53:48 [INFO] Seed set to 1234
2025-08-26 13:54:36 [INFO] Starting Epoch [1/100]
2025-08-26 13:54:55 [INFO] Finished Training Epoch 1: {'Accuracy': '0.8222', 'F1 Score': '0.7997', 'Precision': '0.8248', 'Sensitivity': '0.7761', 'Specificity': '0.8611', 'AUC': '0.8974', 'ECE': '0.0470', 'MCE': '0.0835', 'NLL': 0.40932193, 'Brier': '0.1302', 'Loss': '0.4092', 'Images/s': '497.8507'}
2025-08-26 13:54:55 [INFO] Finished Testing Epoch 1: {'Accuracy': '0.8113', 'F1 Score': '0.7727', 'Precision': '0.8500', 'Sensitivity': '0.7083', 'Specificity': '0.8966', 'AUC': '0.8779', 'ECE': '0.1301', 'MCE': '0.8648', 'NLL': 0.4425099, 'Brier': '0.1401', 'Loss': '0.4423'}
2025-08-26 13:54:56 [INFO] New best model saved with loss 0.4423
2025-08-26 13:54:56 [INFO] Starting Epoch [2/100]
2025-08-26 13:55:15 [INFO] Finished Training Epoch 2: {'Accuracy': '0.8910', 'F1 Score': '0.8792', 'Precision': '0.8914', 'Sensitivity': '0.8674', 'Specificity': '0.9109', 'AUC': '0.9581', 'ECE': '0.0177', 'MCE'

In [9]:
import time
import yaml

seeds = [1234, 5678, 91011, 121314, 151617, 2023, 31415, 27182, 424242, 8675309]

total_runs = len(list_training.keys()) * len(seeds)
run_counter = 0
start_all = time.time()

for j, seed in enumerate(seeds):
    run_counter += 1
    run_start = time.time()

    print(f'[{run_counter}/{total_runs}] | Seed: {seed}')

    config = load_config(path)
    config['path_test'] = f'Datasets\\Folds\\TrainAll\\Test'
    config['path_train'] = f'Datasets\\Folds\\TrainAll\\Train'
    config['soft_label'] = list_training['original']['soft_label']
    config['label_smoothing'] = list_training['original']['label_smoothing']
    config['seed'] = seed
    config['save_dir'] = f'experiments\\ViT_B_32_ENSEMBLE_FINAL\\ensemble_{j}'

    with open(path, 'w') as file:
        yaml.dump(config, file)

    !python train.py --config $path

    run_end = time.time()
    elapsed_run = run_end - run_start
    elapsed_total = run_end - start_all
    avg_per_run = elapsed_total / run_counter
    remaining_time = avg_per_run * (total_runs - run_counter)

    print(f"Run time: {elapsed_run:.2f}s | "
        f"Total elapsed: {elapsed_total/60:.2f} min | "
        f"Estimated remaining: {remaining_time/60:.2f} min\n")

Caching images and labels, please wait...
Caching images and labels, please wait...
Run time: 594.15s | Total elapsed: 89.93 min | Estimated remaining: 9.99 min

[10/10] | Seed: 8675309


2025-08-26 15:19:47 [INFO] Seed set to 424242
2025-08-26 15:20:20 [INFO] Starting Epoch [1/100]
2025-08-26 15:20:35 [INFO] Finished Training Epoch 1: {'Accuracy': '0.7192', 'F1 Score': '0.6613', 'Precision': '0.7373', 'Sensitivity': '0.5995', 'Specificity': '0.8200', 'AUC': '0.7881', 'ECE': '0.0848', 'MCE': '0.1552', 'NLL': 0.5767489, 'Brier': '0.1954', 'Loss': '0.5767', 'Images/s': '660.8787'}
2025-08-26 15:20:35 [INFO] Finished Testing Epoch 1: {'Accuracy': '0.8491', 'F1 Score': '0.8000', 'Precision': '1.0000', 'Sensitivity': '0.6667', 'Specificity': '1.0000', 'AUC': '0.9540', 'ECE': '0.2185', 'MCE': '0.4496', 'NLL': 0.46658683, 'Brier': '0.1457', 'Loss': '0.4411'}
2025-08-26 15:20:35 [INFO] New best model saved with loss 0.4411
2025-08-26 15:20:35 [INFO] Starting Epoch [2/100]
2025-08-26 15:20:50 [INFO] Finished Training Epoch 2: {'Accuracy': '0.7740', 'F1 Score': '0.7354', 'Precision': '0.7915', 'Sensitivity': '0.6867', 'Specificity': '0.8476', 'AUC': '0.8484', 'ECE': '0.0861', 'MC

Caching images and labels, please wait...
Caching images and labels, please wait...
Run time: 677.98s | Total elapsed: 101.23 min | Estimated remaining: 0.00 min



2025-08-26 15:29:42 [INFO] Seed set to 8675309
2025-08-26 15:30:15 [INFO] Starting Epoch [1/100]
2025-08-26 15:30:31 [INFO] Finished Training Epoch 1: {'Accuracy': '0.7161', 'F1 Score': '0.6583', 'Precision': '0.7322', 'Sensitivity': '0.5979', 'Specificity': '0.8157', 'AUC': '0.7903', 'ECE': '0.0854', 'MCE': '0.1543', 'NLL': 0.57712936, 'Brier': '0.1954', 'Loss': '0.5772', 'Images/s': '624.9964'}
2025-08-26 15:30:31 [INFO] Finished Testing Epoch 1: {'Accuracy': '0.8302', 'F1 Score': '0.7692', 'Precision': '1.0000', 'Sensitivity': '0.6250', 'Specificity': '1.0000', 'AUC': '0.9454', 'ECE': '0.1827', 'MCE': '0.4522', 'NLL': 0.47389218, 'Brier': '0.1488', 'Loss': '0.4392'}
2025-08-26 15:30:31 [INFO] New best model saved with loss 0.4392
2025-08-26 15:30:31 [INFO] Starting Epoch [2/100]
2025-08-26 15:30:46 [INFO] Finished Training Epoch 2: {'Accuracy': '0.7704', 'F1 Score': '0.7347', 'Precision': '0.7787', 'Sensitivity': '0.6954', 'Specificity': '0.8335', 'AUC': '0.8462', 'ECE': '0.0802', '